In [1]:
from  Controller import TrainingController_V2, ExplainingController_V2
from Parameters.PredictingParameters import PredictingParameters
from Parameters.TrainingParameters import TrainingParameters
from Parameters.Enums import SelectableLoss, SelectableModels, SelectableDatasets
import torch
import pyAgrum.lib.notebook as gnb
from IPython.core.display import display, HTML
from Utils.SaveUtils import load_parameters
import json

/Users/jrhs/.pyenv/versions/3.9.4/envs/dl/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
######################## Resume training ########################
folder_path = "./SavedModels/" + "0.8597_BPI2012_BaseLineLSTMModel_2021-05-13 19:32:33.129899"
parameters_json = load_parameters(folder_path=folder_path)
parameters = TrainingParameters(**parameters_json)
parameters.load_model_folder_path = folder_path
predicting_parameters = PredictingParameters()
predicting_parameters.load_model_folder_path = folder_path

In [3]:
trainer = TrainingController_V2(parameters=parameters)
explainer = ExplainingController_V2(parameters=parameters, predicting_parameters= predicting_parameters)


| Running on cpu  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012/AOW 

| Model loaded successfully from: ./SavedModels/0.8597_BPI2012_BaseLineLSTMModel_2021-05-13 19:32:33.129899  

| Model loaded successfully from ./SavedModels/0.8597_BPI2012_BaseLineLSTMModel_2021-05-13 19:32:33.129899/model.pt 
/Users/jrhs/.pyenv/versions/3.9.4/envs/dl/lib/python3.9/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<Figure size 1600x800 with 0 Axes>

In [4]:
########### Get example data from trainer ###########
print("Test set length: %d" %(len(trainer.test_dataset)))
index_from_test = 45

Test set length: 1308


In [5]:
example_trace = trainer.model.vocab.list_of_index_to_vocab(trainer.test_dataset[index_from_test]["trace"])

In [6]:
explainer.show_model_info()


| Model Structure 
BaselineLSTMModel_V2(
  (emb): Embedding(39, 32, padding_idx=0)
  (lstm): LSTM(32, 64, batch_first=True, dropout=0.1)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output_net): Sequential(
    (0): LeakyReLU(negative_slope=0.01, inplace=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=64, out_features=39, bias=True)
  )
)
| Loaded model has {29127} parameters 


In [7]:
df, data_predicted_list, bn, bn_html, inference, infoBN, markov_blanket_html = explainer.pm_predict_lindaBN_explain(example_trace)


| Selecting Greedy Hill Climbing Algorithm 
/Users/jrhs/Documents/GitHub/Apply-LINDA-BN-on-predicting-next-event/Utils/VocabDict.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sorted_seq_list = [torch.tensor(seq_list[idx])


In [8]:
generated_html = explainer.generate_html_page_from_graphs("<pre>" +json.dumps(example_trace)+"</pre>", "<pre>" + json.dumps(data_predicted_list) + "</pre>", bn_html, inference, infoBN, markov_blanket_html)

In [9]:
display(HTML(generated_html))

In [10]:
explainer.save_html(html_content= generated_html)


| HTML page has been saved to: ./Explanations/0.8597_BPI2012_BaseLineLSTMModel_2021-05-13 19:32:33.129899_2021-05-15 01:58:48.846973_graphs_LINDA-BN.html 


In [11]:
display(HTML(json.dumps(data_predicted_list)))

In [12]:
data_predicted_list

['<SOS>',
 'A_SUBMITTED_COMPLETE',
 'A_PARTLYSUBMITTED_COMPLETE',
 'A_DECLINED_COMPLETE',
 '<EOS>',
 '<EOS>']